In [120]:
# Required packages: ntlk, sklearn+pandas
# !pip install nltk
# import nltk
# nltk.download()
# !pip install pandas
# !pip install sklearn
# !pip install beautifulsoup4
# !pip install contractions

## TUT2_Q4
Write a program to do the following tasks:
1. Download the Web page of a given link and extract the text content of the
page
2. Split the text into sentences and count sentences
3. Split the text into tokens and count token types
4. Find lemmas (or stems) of the tokens and count lemma types
5. Do stemming on the tokens and count unique "stemmed" tokens
### Task1: Download text content of page


In [121]:
from urllib.request import urlopen
url="https://en.wikipedia.org/wiki/Strom_Thurmond_filibuster_of_the_Civil_Rights_Act_of_1957"

page=urlopen(url)
page

In [122]:
html_bytes = page.read()
html = html_bytes.decode("utf-8")

In [123]:
print(html[:300])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Strom Thurmond filibuster of the Civil Rights Act of 1957 - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],


This is just a bunch of a html codes, doesn't really make sense. Try extract using external libraries

In [124]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, features="html.parser") # printing soup gives out same result as html print
for script in soup(['script','style']):
    script.extract()
text = soup.get_text()

### Task2: Split the text into sentences and count sentences

In [125]:
#one option is to simply split by '.'
a = text.split('.') # split sentences by stop mark
print(a[:15])

['\n\n\n\nStrom Thurmond filibuster of the Civil Rights Act of 1957 - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nStrom Thurmond filibuster of the Civil Rights Act of 1957\n\nFrom Wikipedia, the free encyclopedia\n\n\n\nJump to navigation\nJump to search\nLongest United States Senate filibuster\n\n\n Strom Thurmond, c', '\u20091961\n\n\n\nWikisource has original text related to this article:\nStrom Thurmond filibuster on the Civil Rights Act of 1957\n\nOn August 28, 1957, Strom Thurmond, a United States senator from South Carolina, began a filibuster intended to prevent the passage of the Civil Rights Act of 1957', ' The filibuster, an extended speech designed to stall legislation, began at 8:54\xa0p', 'm', '[Note 1] and lasted until 9:12\xa0p', 'm', ' the following day, a duration of 24 hours and 18 minutes', ' This made the filibuster the longest single-person filibuster in United States Senate history, a record that still stands a

Can see from the result that there is many marks and not always accurate. Another way to do it is to use external tools

In [126]:
# use BeautifulSoup
# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
chunks = [chunk for chunk in chunks if chunk]
print(chunks[:10])
print(len(chunks))

['Strom Thurmond filibuster of the Civil Rights Act of 1957 - Wikipedia', 'Strom Thurmond filibuster of the Civil Rights Act of 1957', 'From Wikipedia, the free encyclopedia', 'Jump to navigation', 'Jump to search', 'Longest United States Senate filibuster', 'Strom Thurmond, c.\u20091961', 'Wikisource has original text related to this article:', 'Strom Thurmond filibuster on the Civil Rights Act of 1957', 'On August 28, 1957, Strom Thurmond, a United States senator from South Carolina, began a filibuster intended to prevent the passage of the Civil Rights Act of 1957. The filibuster, an extended speech designed to stall legislation, began at 8:54\xa0p.m.[Note 1] and lasted until 9:12\xa0p.m. the following day, a duration of 24 hours and 18 minutes. This made the filibuster the longest single-person filibuster in United States Senate history, a record that still stands as of 2022[update]. The filibuster focused primarily on asserting that the bill was both unnecessary and unconstitution

### Task3: Split the text into tokens and count token types
1. concatenate all chunks into 1 long string
2. string cleaning

In [127]:
text='\n'.join(chunks)
print(text[:300], '\n')
print("Length of all text:", len(text))

Strom Thurmond filibuster of the Civil Rights Act of 1957 - Wikipedia
Strom Thurmond filibuster of the Civil Rights Act of 1957
From Wikipedia, the free encyclopedia
Jump to navigation
Jump to search
Longest United States Senate filibuster
Strom Thurmond, c. 1961
Wikisource has original text related 

Length of all text: 18375


In [128]:
import re
def cleanEmoji(text:str):
    emoji_pattern = re.compile("["
                           u"U0001F600-U0001F64F"  # emoticons
                           u"U0001F300-U0001F5FF"  # symbols & pictographs
                           u"U0001F680-U0001F6FF"  # transport & map symbols
                           u"U0001F1E0-U0001F1FF"  # flags (iOS)
                           u"U00002702-U000027B0"
                           u"U000024C2-U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    # remove emoticons
    EMOTICONS = {
        u":‑)":"Happy face or smiley",
        u":)":"Happy face or smiley",
        u":-]":"Happy face or smiley",
        u":]":"Happy face or smiley",
        u":-3":"Happy face smiley",
        u":3":"Happy face smiley",
        u":->":"Happy face smiley",
        u":>":"Happy face smiley",
        u"8-)":"Happy face smiley",
        u":o)":"Happy face smiley",
        u":-}":"Happy face smiley",
        u":}":"Happy face smiley",
        u":-)":"Happy face smiley",
        u":c)":"Happy face smiley",
        u":^)":"Happy face smiley",
        u"=]":"Happy face smiley"
    }
    ans = re.compile(u'(" + u"|".join(k for k in EMOTICONS) + u")')
    ans = ans.sub(r'',text)

In [129]:
from string import punctuation
import contractions
def cleanBasics(text:str):
    #.translate(str.maketrans(",", punctuation)) #  removing punctations: "!"#$%&\'()*+,-./:;?@[\\]^_{|}~`"
    text=''.join([i for i in text if (not i.isdigit()) and (not i in punctuation)]) # removing numbers 
    text=" ".join(text.split())# removing extra spaces
    text=contractions.fix(text)
    return text

In [130]:
### utility function for counting word types
def cntWordOccurence(word_list:list):
    word_cnt = {}
    for uni_word in set(word_list):
        word_cnt[uni_word]=word_list.count(uni_word)
    word_cnt_sorted = {k: v for k, v in sorted(word_cnt.items(), key=lambda item: item[1], reverse=True)}
    return word_cnt_sorted

In [131]:
from itertools import islice

def takeFirstN(iterable:dict, n:int):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

def printFirstN(iterable:dict,n):
    i=0
    for k,v in iterable.items():
        print(k,v)
        i+=0
        if i == n:
            return

In [132]:
text=cleanBasics(text)

In [133]:
from nltk.tokenize import word_tokenize
word_list=word_tokenize(text)
word_cnt_sorted = cntWordOccurence(word_list)
word_cnt_sorted

{'the': 137,
 'of': 80,
 'a': 77,
 'and': 64,
 'to': 64,
 'Thurmond': 50,
 'filibuster': 41,
 'in': 39,
 'by': 30,
 'was': 27,
 'The': 26,
 'Senate': 23,
 'as': 22,
 'for': 22,
 'p': 20,
 'Retrieved': 20,
 'bill': 19,
 'Thurmonds': 18,
 'from': 18,
 'his': 17,
 'Rights': 17,
 'on': 17,
 'that': 17,
 'Strom': 16,
 'had': 16,
 'United': 16,
 'Act': 15,
 'Civil': 15,
 'States': 15,
 'March': 14,
 'speech': 13,
 'Southern': 13,
 'b': 12,
 'at': 12,
 'senators': 11,
 'August': 11,
 'Crespino': 10,
 'been': 10,
 'c': 10,
 'vote': 10,
 'but': 9,
 'South': 9,
 'Lachicotte': 9,
 'he': 8,
 'against': 8,
 'hours': 8,
 'with': 8,
 '–': 8,
 'him': 8,
 'it': 8,
 'not': 8,
 'rights': 7,
 'Cohodas': 7,
 'an': 7,
 'which': 6,
 'this': 6,
 'also': 6,
 'allowed': 6,
 'time': 6,
 'after': 6,
 'right': 6,
 'has': 5,
 'civil': 5,
 'This': 5,
 'restroom': 5,
 'states': 5,
 'other': 5,
 'Record': 5,
 'pp': 5,
 'is': 5,
 'trial': 5,
 'Carolina': 5,
 'Americans': 5,
 'passed': 5,
 'US': 5,
 'be': 5,
 'York': 5,

Tokenization and count and be done simultaneously using sklearn-vectorizer

In [134]:
import pandas as pd
df = pd.DataFrame({'text':[text]})

In [135]:
from sklearn.feature_extraction.text import CountVectorizer
# initialize
cv = CountVectorizer(stop_words='english') 
cv_matrix = cv.fit_transform(df['text']) 
# create document term matrix
df_dtm = pd.DataFrame(cv_matrix.toarray(), columns=cv.get_feature_names())
df_dtm

c:\Users\ytchen\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,able,absorb,accessible,according,accountlog,act,adam,added,additional,address,...,worked,worn,writein,written,year,years,yield,yielded,york,한국어
0,1,1,1,2,1,15,1,1,1,1,...,2,1,1,1,2,4,1,1,5,1


### Tesk4: Find lemmas (or stems) of the tokens and count lemma types

In [140]:
PRINT_FIRST=15

In [142]:
from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()
word_lemma=[lemmatizer.lemmatize(x) for x in word_list]
lemma_occur_cnt=cntWordOccurence(word_lemma)
print("Lemma found using nltk:",len(lemma_occur_cnt))
print("Top {x} items are: ".format(x=PRINT_FIRST))
printFirstN(lemma_occur_cnt, PRINT_FIRST)

Lemma found using nltk: 936
Top 15 items are: 
the 137
a 99
of 80
and 64
to 64
Thurmond 50
filibuster 42
in 39
by 30
wa 27
The 26
Senate 23
for 22
p 20
Retrieved 20
bill 20
Thurmonds 18
from 18
his 17
Rights 17
on 17
that 17
Strom 16
had 16
United 16
senator 16
Act 15
Civil 15
States 15
March 14
speech 13
Southern 13
right 13
b 12
at 12
August 11
vote 11
Crespino 10
been 10
c 10
but 9
South 9
it 9
Lachicotte 9
hour 9
he 8
against 8
with 8
– 8
him 8
not 8
Cohodas 7
time 7
an 7
which 6
this 6
also 6
state 6
allowed 6
after 6
year 6
ha 5
civil 5
This 5
restroom 5
other 5
Record 5
pp 5
is 5
law 5
trial 5
Carolina 5
Americans 5
passed 5
US 5
be 5
York 5
Eisenhower 5
himself 5
Wikipedia 5
New 5
African 5
jury 5
provision 5
ISBN 5
made 4
would 4
Democrats 4
out 4
longest 4
prevent 4
record 4
American 4
Filibuster 4
case 4
June 4
Thompson 4
pm 4
two 4
cloture 4
Washington 4
D 4
before 4
into 4
minute 4
Bass 4
more 4
received 4
Joseph 4
described 4
if 4
using 4
February 4
Russell 4
were 4
Dwigh

### Task5: Do stemming on the tokens and count unique "stemmed" tokens

In [117]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

porter = PorterStemmer()
lancaster=LancasterStemmer()

In [139]:
from itertools import islice
word_porter_stem=[porter.stem(x) for x in word_list]
word_lancaster_stem=[lancaster.stem(x) for x in word_list]
porter_occur_cnt=cntWordOccurence(word_porter_stem)
lanca_occur_cnt=cntWordOccurence(word_lancaster_stem)
print("Stem found using porter:",len(porter_occur_cnt))
print("Top {x} items are: ".format(x=PRINT_FIRST))
printFirstN(porter_occur_cnt, PRINT_FIRST)
print("Stem found using lancaster:",len(lanca_occur_cnt))
print("Top {x} items are: ".format(x=PRINT_FIRST))
printFirstN(lanca_occur_cnt, PRINT_FIRST)

Stem found using porter: 798
Top 20 items are: 
the 163
of 80
a 79
thurmond 68
and 64
to 64
filibust 49
senat 45
in 42
by 31
right 30
wa 27
as 24
state 23
for 22
civil 20
retriev 20
p 20
bill 20
on 19
from 19
that 18
hi 18
southern 16
had 16
strom 16
unit 16
at 15
act 15
vote 15
march 14
speech 13
b 12
august 11
thi 11
he 11
c 11
been 10
south 10
record 10
crespino 10
it 10
hour 10
but 9
an 9
allow 9
lachicott 9
american 9
not 9
use 8
democrat 8
against 8
time 8
with 8
– 8
him 8
new 8
d 7
elect 7
cohoda 7
end 6
which 6
also 6
articl 6
pass 6
parti 6
is 6
after 6
be 6
year 6
polit 6
ha 5
russel 5
longest 5
prevent 5
restroom 5
provis 5
juri 5
isbn 5
other 5
person 5
eisenhow 5
pp 5
african 5
law 5
trial 5
more 5
us 5
dure 5
himself 5
clotur 5
about 5
who 5
york 5
wikipedia 5
carolina 5
washington 5
last 4
serv 4
made 4
would 4
thompson 4
sign 4
out 4
speak 4
later 4
dwight 4
did 4
leav 4
receiv 4
date 4
case 4
pm 4
two 4
into 4
joseph 4
befor 4
mani 4
if 4
were 4
februari 4
june 4
racis